In [1]:
# Importando as bibliotecas necessarias
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib notebook
import pandas as pd
from scipy.stats import mode, pearsonr, spearmanr
import glob, os
pd.set_option("display.max_rows", 300)

In [2]:
def vote_acc(selected_nets):

    all_predicted = np.empty([selected_nets.shape[0], 63])
    for net_number in range(selected_nets.shape[0]):
        netX = selected_nets.iloc[net_number]
        net_predicted_str = netX['test_predicted'][1:-1].replace("\n", "")
        net_predicted = np.fromstring(net_predicted_str, dtype=int, sep=' ')
        all_predicted[net_number] = net_predicted

    print(all_predicted.shape)

    voteClassOut, count = mode(all_predicted, axis=0)
    # print(voteClassOut)
    voteClassOut = voteClassOut.reshape(63,1)

    classTarget = np.load('y_test_class.npy')
    # print(classTarget.reshape(1, 63))

    print('Acurácia', np.sum(classTarget.squeeze()==voteClassOut.squeeze())/len(classTarget)*100, '%')
#     plt.figure()
#     plt.plot(classTarget, 'ob')
#     plt.plot(voteClassOut, 'xr')
#     plt.show()

In [3]:
def vote_acc_8out(selected_nets):

    all_predicted = np.empty([selected_nets.shape[0], 55])
    for net_number in range(selected_nets.shape[0]):
        netX = selected_nets.iloc[net_number]
        net_predicted_str = netX['test_predicted'][1:-1].replace("\n", "")
        net_predicted = np.fromstring(net_predicted_str, dtype=int, sep=' ')
        all_predicted[net_number] = net_predicted[8:63]

    print(all_predicted.shape)

    voteClassOut, count = mode(all_predicted, axis=0)
    # print(voteClassOut)
    voteClassOut = voteClassOut.reshape(55,1)

    classTarget = np.load('y_test_class.npy')
    classTarget_8out = classTarget[8:63]
    # print(classTarget.reshape(1, 63))

    print('Acurácia 8out', np.sum(classTarget_8out.squeeze()==voteClassOut.squeeze())/len(classTarget_8out)*100, '%')
#     plt.figure()
#     plt.plot(classTarget, 'ob')
#     plt.plot(voteClassOut, 'xr')
#     plt.show()

In [4]:
def test_acc_8out(selected_nets):

    all_predicted = np.empty([selected_nets.shape[0], 8])
    for net_number in range(selected_nets.shape[0]):
        netX = selected_nets.iloc[net_number]
        net_predicted_str = netX['test_predicted'][1:-1].replace("\n", "")
        net_predicted = np.fromstring(net_predicted_str, dtype=int, sep=' ')
        all_predicted[net_number] = net_predicted[0:8]

    print(all_predicted.shape)

    voteClassOut, count = mode(all_predicted, axis=0)
    # print(voteClassOut)
    voteClassOut = voteClassOut.reshape(8,1)

    classTarget = np.load('y_test_class.npy')
    classTarget_8out = classTarget[0:8]

    print('Acurácia test 8out', np.sum(classTarget_8out.squeeze()==voteClassOut.squeeze())/len(classTarget_8out)*100, '%')
#     plt.figure()
#     plt.plot(classTarget, 'ob')
#     plt.plot(voteClassOut, 'xr')
#     plt.show()

In [5]:
# Correlação Acc_vali x Acc_test
def acc_corr(selected_nets, ft_vali, ft_test):
    feature_vali = selected_nets[ft_vali]
    feature_test = selected_nets[ft_test]
    pcorr, _ = pearsonr(feature_vali, feature_test)
    scorr, _ = spearmanr(feature_vali, feature_test)
    print(str(ft_vali),'x',str(ft_test))
    print('Correlação de Pearson:', pcorr)
    print('Correlação de Spearman:', scorr)
#     plt.scatter(feature_vali, feature_test)
#     plt.xlabel(ft_vali)
#     plt.ylabel(ft_test)
#     plt.show()

In [6]:
# # Juntar todos os xlsx em um só
# all_data = pd.DataFrame()
# for f in glob.glob('D:/TG-HPC_data/hpc_xlsx/*.xlsx'):
#     df = pd.read_excel(f)
#     all_data = all_data.append(df,ignore_index=True)
# print(all_data)
# all_data.to_excel ('all_nets_hpc.xlsx', index = False, header=True)

In [7]:
# Read xlsx with nets info
df_nets = pd.read_excel('all_nets_hpc.xlsx', index_col=None, header=0)
# df_nets = pd.read_excel('./Loop_v1/classification_nets_v1.xlsx', index_col=None, header=0)
# vote_acc(df_nets)

In [8]:
all_predicted = np.empty([df_nets.shape[0], 63])
new_acc = np.empty([df_nets.shape[0]])
classTarget = np.load('y_test_class.npy')
classTarget_8out = classTarget[8:63]
for net_number in range(df_nets.shape[0]):
    netX = df_nets.iloc[net_number]
    net_predicted_str = netX['test_predicted'][1:-1].replace("\n", "")
    net_predicted = np.fromstring(net_predicted_str, dtype=int, sep=' ')
    net_predicted_8out = net_predicted[8:63]
    all_predicted[net_number] = net_predicted  
    
    new_acc[net_number] = np.sum(classTarget_8out.squeeze()==net_predicted_8out.squeeze())/len(classTarget_8out)*100

In [9]:
df_nets['new_acc'] = new_acc
max(new_acc)

49.09090909090909

In [10]:
df_nets['new_acc']

0        32.727273
1        20.000000
2        20.000000
3        23.636364
4        32.727273
           ...    
47305    25.454545
47306    32.727273
47307    23.636364
47308    30.909091
47309    21.818182
Name: new_acc, Length: 47310, dtype: float64

In [22]:
# Retira linhas em que a nova acurácia foi menor que o valor estabelecido
df_nets_vali = df_nets.drop(np.where(df_nets['new_acc']<47)[0], axis=0)
df_nets_vali.reset_index(drop=True, inplace=True)
vote_acc_8out(df_nets_vali)

(13, 55)
Acurácia 8out 61.81818181818181 %


In [12]:
df_nets_vali.to_excel ('selected_nets_new_acc50_8out.xlsx', index = False, header=True)

In [13]:
# Read xlsx with nets info
selected_nets_new_acc50_8out = pd.read_excel('selected_nets_new_acc50_8out.xlsx', index_col=None, header=0)

In [14]:
selected_nets_new_acc50_8out

,Net,random_state,num_layers,layers_size,net_param,criterion,learning_rate,optimizer,epochs,vali_best_epoch,...,vali_sur_R,vali_c_matrix,vali_c_matrix_perc,test_acc,test_loss,test_R,test_predicted,test_c_matrix,test_c_matrix_perc,new_acc
0,199,82,4,"[4, 8, 4]",<bound method Module.parameters of Net(\n (li...,CrossEntropyLoss,0.022,Adam,50000,7900,...,0.107848,[[3 0 0 0]\n [1 4 4 1]\n [2 3 2 2]\n [0 0 1 6]],[[100. 0. 0. 0. ...,44,39.951607,0.459575,[0 0 3 1 0 3 2 0 1 2 3 0 0 3 2 3 0 3 0 3 0 3 3...,[[16 2 2 3]\n [ 5 4 3 3]\n [ 3 1 3 6]...,[[69.56521739 8.69565217 8.69565217 13.04347...,47.272727
1,18506,253,4,"[11, 9, 4]",<bound method Module.parameters of Net(\n (li...,CrossEntropyLoss,0.070,Adam,50000,200,...,-0.168544,[[3 2 0 2]\n [1 3 0 4]\n [1 1 0 4]\n [1 3 0 4]],[[42.85714286 28.57142857 0. 28.57142...,42,5.879779,0.389403,[1 3 3 1 3 1 3 1 1 3 0 0 2 3 3 3 1 3 0 3 0 3 1...,[[ 9 6 0 8]\n [ 1 7 2 5]\n [ 4 1 0 8]...,[[39.13043478 26.08695652 0. 34.78260...,47.272727
2,20812,5583,4,"[21, 15, 4]",<bound method Module.parameters of Net(\n (li...,CrossEntropyLoss,0.027,Adam,50000,39600,...,-0.006464,[[5 0 1 0]\n [3 3 1 1]\n [3 3 0 0]\n [1 2 2 4]],[[83.33333333 0. 16.66666667 0. ...,44,71.437027,0.421138,[0 1 3 0 3 0 3 3 1 2 0 1 3 1 0 2 0 1 0 1 0 2 2...,[[12 5 1 5]\n [ 4 3 4 4]\n [ 2 3 5 3]...,[[52.17391304 21.73913043 4.34782609 21.73913...,47.272727
3,21241,6080,4,"[4, 22, 4]",<bound method Module.parameters of Net(\n (li...,CrossEntropyLoss,0.038,Adam,50000,2700,...,-0.255434,[[4 2 1 3]\n [2 2 1 2]\n [3 1 1 1]\n [1 2 0 3]],[[40. 20. 10. 30. ...,47,13.260184,0.396985,[0 0 0 0 2 0 3 0 0 3 1 0 2 0 3 3 0 3 0 3 0 0 0...,[[16 2 3 2]\n [ 7 3 1 4]\n [ 5 1 4 3]...,[[69.56521739 8.69565217 13.04347826 8.69565...,47.272727
4,27190,2445,4,"[3, 18, 4]",<bound method Module.parameters of Net(\n (li...,CrossEntropyLoss,0.047,Adam,50000,1000,...,-0.097047,[[7 0 2 0]\n [5 1 1 0]\n [3 2 3 1]\n [1 1 1 1]],[[77.77777778 0. 22.22222222 0. ...,47,13.004984,0.422357,[3 0 3 1 0 0 1 1 1 2 0 2 0 1 1 0 0 2 1 3 0 2 0...,[[14 5 1 3]\n [ 8 5 1 1]\n [ 3 1 6 3]...,[[60.86956522 21.73913043 4.34782609 13.04347...,47.272727
5,28984,8223,4,"[8, 9, 4]",<bound method Module.parameters of Net(\n (li...,CrossEntropyLoss,0.047,Adam,50000,200,...,-0.141170,[[2 0 1 0]\n [3 4 3 0]\n [1 0 4 3]\n [2 0 3 3]],[[66.66666667 0. 33.33333333 0. ...,46,18.098665,0.471912,[1 2 0 0 2 3 2 1 1 1 1 0 2 2 2 0 2 2 0 0 0 2 1...,[[11 2 9 1]\n [ 3 6 6 0]\n [ 2 3 7 1]...,[[47.82608696 8.69565217 39.13043478 4.34782...,47.272727
6,29059,1170,4,"[12, 11, 4]",<bound method Module.parameters of Net(\n (li...,CrossEntropyLoss,0.068,Adam,50000,1700,...,0.136046,[[4 5 1 0]\n [4 2 1 2]\n [2 1 1 0]\n [0 1 2 3]],[[40. 50. 10. 0. ...,46,53.844082,0.332687,[3 0 0 1 0 2 3 0 1 2 1 0 0 0 1 2 0 2 0 3 1 3 1...,[[11 7 3 2]\n [ 4 7 1 3]\n [ 4 1 6 2]...,[[47.82608696 30.43478261 13.04347826 8.69565...,47.272727
7,31773,2078,4,"[21, 4, 4]",<bound method Module.parameters of Net(\n (li...,CrossEntropyLoss,0.034,Adam,50000,500,...,0.179166,[[4 0 1 0]\n [9 1 1 0]\n [5 1 0 0]\n [3 1 0 3]],[[80. 0. 20. 0. ...,49,8.416088,0.479702,[0 0 3 0 0 0 2 0 0 1 0 0 0 0 0 1 0 2 0 3 0 2 0...,[[19 1 1 2]\n [11 1 3 0]\n [ 5 1 6 1]...,[[82.60869565 4.34782609 4.34782609 8.69565...,49.090909
8,40078,3728,4,"[20, 19, 4]",<bound method Module.parameters of Net(\n (li...,CrossEntropyLoss,0.001,Adam,50000,500,...,0.066660,[[3 2 5 0]\n [2 4 4 1]\n [0 0 2 0]\n [1 1 1 3]],[[ 30. 20. 50. 0. ...,44,4.141557,0.408773,[1 2 2 0 1 2 2 1 1 2 2 0 1 1 1 3 1 2 0 3 0 2 0...,[[7 9 5 2]\n [2 7 4 2]\n [2 1 8 2]\n [0 4 2 6]],[[30.43478261 39.13043478 21.73913043 8.69565...,47.272727
9,42253,9315,4,"[19, 7, 4]",<bound method Module.parameters of Net(\n (li...,CrossEntropyLoss,0.055,Adam,50000,13700,...,0.294552,[[4 0 0 3]\n [2 7 0 2]\n [0 0 1 4]\n [1 2 0 3]],[[57.14285714 0. 0. 42.85714...,44,1.715671,0.489436,[3 0 3 0 1 3 3 1 1 3 1 1 1 3 3 3 1 3 0 3 0 3 1...,[[10 5 1 7]\n [ 2 7 0 6]\n [ 0 5 0 8]...,[[43.47826087 21.73913043 4.34782609 30.43478...,47.2727

In [15]:
test_acc_8out(selected_nets_new_acc50_8out)

(13, 8)
Acurácia test 8out 50.0 %


# Adaptar para retirar 8 amostras aleatórias do total
# MELHOR SELEÇÃO DE REDES

In [40]:
# Read xlsx with nets info
df_nets = pd.read_excel('all_nets_hpc.xlsx', index_col=None, header=0)

In [45]:
leave_out = np.random.randint(0,63,8)
print(leave_out)
all_patients = range(63)
selected_patients = list(set(all_patients) - set(leave_out))

[21 20 23 55  5 41 60 31]


# leave_out = [21 20 23 55  5 41 60 31]

In [46]:
len(selected_patients)

55

In [47]:
all_predicted = np.empty([df_nets.shape[0], 55])
new_acc = np.empty([df_nets.shape[0]])
classTarget = np.load('y_test_class.npy')
classTarget_8out = classTarget[selected_patients]
for net_number in range(df_nets.shape[0]):
    netX = df_nets.iloc[net_number]
    net_predicted_str = netX['test_predicted'][1:-1].replace("\n", "")
    net_predicted = np.fromstring(net_predicted_str, dtype=int, sep=' ')
    net_predicted_8out = net_predicted[selected_patients]
    all_predicted[net_number] = net_predicted[selected_patients]
    
    new_acc[net_number] = np.sum(classTarget_8out.squeeze()==net_predicted_8out.squeeze())/len(classTarget_8out)*100

df_nets['new_acc'] = new_acc
print(max(new_acc))

47.27272727272727


In [59]:
# Retira linhas em que a nova acurácia foi menor que o valor estabelecido
df_nets_vali = df_nets.drop(np.where(df_nets['new_acc']<45)[0], axis=0)
df_nets_vali.reset_index(drop=True, inplace=True)
    
all_predicted = np.empty([df_nets_vali.shape[0], 55])
for net_number in range(df_nets_vali.shape[0]):
    netX = df_nets_vali.iloc[net_number]
    net_predicted_str = netX['test_predicted'][1:-1].replace("\n", "")
    net_predicted = np.fromstring(net_predicted_str, dtype=int, sep=' ')
    all_predicted[net_number] = net_predicted[selected_patients]

print(all_predicted.shape)

voteClassOut, count = mode(all_predicted, axis=0)
# print(voteClassOut)
voteClassOut = voteClassOut.reshape(55,1)

classTarget = np.load('y_test_class.npy')
classTarget_8out = classTarget[selected_patients]
# print(classTarget.reshape(1, 63))

print('Acurácia 8out', np.sum(classTarget_8out.squeeze()==voteClassOut.squeeze())/len(classTarget_8out)*100, '%')

df_nets_vali.to_excel ('selected_nets_new_acc_8out.xlsx', index = False, header=True)

(10, 55)
Acurácia 8out 56.36363636363636 %


In [60]:
# Read xlsx with nets info
selected_nets_new_acc_8out = pd.read_excel('selected_nets_new_acc_8out.xlsx', index_col=None, header=0)

all_predicted = np.empty([selected_nets_new_acc_8out.shape[0], 8])
for net_number in range(selected_nets_new_acc_8out.shape[0]):
    netX = selected_nets_new_acc_8out.iloc[net_number]
    net_predicted_str = netX['test_predicted'][1:-1].replace("\n", "")
    net_predicted = np.fromstring(net_predicted_str, dtype=int, sep=' ')
    all_predicted[net_number] = net_predicted[leave_out]

print(all_predicted.shape)

voteClassOut, count = mode(all_predicted, axis=0)
# print(voteClassOut)
voteClassOut = voteClassOut.reshape(8,1)

classTarget = np.load('y_test_class.npy')
classTarget_8out = classTarget[leave_out]

print('Acurácia test 8out', np.sum(classTarget_8out.squeeze()==voteClassOut.squeeze())/len(classTarget_8out)*100, '%')

(10, 8)
Acurácia test 8out 62.5 %


In [61]:
classTarget_8out

array([[2],
       [0],
       [3],
       [3],
       [0],
       [0],
       [3],
       [2]], dtype=int64)

In [62]:
voteClassOut

array([[3.],
       [0.],
       [2.],
       [0.],
       [0.],
       [0.],
       [3.],
       [2.]])